<a href="https://colab.research.google.com/github/theLabro/Simple-To-Do-List/blob/main/pds_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!ls

dataset2.csv  dataset.csv  features.py	__pycache__  sample_data


In [3]:
!pip install pandas numpy scikit-learn tldextract joblib

In [4]:
import pandas as pd
import joblib

from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [5]:
df = pd.read_csv("dataset2.csv")

df.head()

,url,label (0:Legitimate || 1: Phishing)
0,https://www.google.com,0
1,https://www.youtube.com,0
2,https://www.facebook.com,0
3,https://www.baidu.com,0
4,https://www.wikipedia.org,0


In [6]:
df.columns

Index(['url', 'label (0:Legitimate || 1: Phishing)'], dtype='object')

In [7]:
df.head()

,url,label (0:Legitimate || 1: Phishing)
0,https://www.google.com,0
1,https://www.youtube.com,0
2,https://www.facebook.com,0
3,https://www.baidu.com,0
4,https://www.wikipedia.org,0


In [8]:
df.rename(columns={'label (0:Legitimate || 1: Phishing)': 'label'}, inplace=True)

In [9]:
df.columns

Index(['url', 'label'], dtype='object')

In [10]:
df['label'].value_counts()

,count
label,
0,345738
1,104438


In [11]:
from features import url_features

X_dict = df['url'].apply(url_features).tolist()
y = df['label']

In [12]:
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer(sparse=False)
X = vectorizer.fit_transform(X_dict)

X.shape

(450176, 13)

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [14]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42)

In [15]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9688346883468835

Confusion Matrix:
 [[68613   535]
 [ 2271 18617]]

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98     69148
           1       0.97      0.89      0.93     20888

    accuracy                           0.97     90036
   macro avg       0.97      0.94      0.95     90036
weighted avg       0.97      0.97      0.97     90036



In [16]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5)
scores, scores.mean()

(array([0.93312675, 0.97649803, 0.89350808, 0.94183373, 0.91340034]),
 np.float64(0.9316733867346046))

In [17]:
import joblib

joblib.dump(model, "phishing_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

['vectorizer.pkl']

In [18]:
model = joblib.load("phishing_model.pkl")
vectorizer = joblib.load("vectorizer.pkl")

In [26]:
def predict_url(url):
    feats = url_features(url)
    X_new = vectorizer.transform([feats])
    pred = model.predict(X_new)[0]
    return "Phishing 🚨" if pred == 1 else "Legitimate ✅"


print(predict_url(""))
print(predict_url(""))
print(predict_url(""))
print(predict_url(""))

Phishing 🚨
Legitimate ✅
Legitimate ✅
Phishing 🚨


In [21]:
from sklearn.metrics import accuracy_score

# Predict on test data
y_pred = model.predict(X_test)

# Calculate accuracy
acc = accuracy_score(y_test, y_pred)

print("Test Accuracy:", acc)

Test Accuracy: 0.9688346883468835


In [22]:
print("Test Accuracy: {:.2f}%".format(acc * 100))

Test Accuracy: 96.88%


In [23]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))

[[68613   535]
 [ 2271 18617]]
